You can create a service using the CLI commands below.
This practice session will follow the screenshots provided on p.47-p.55 of [this document](http://azure.studydev.com/openai/aoai_2024_pe_02.pdf) to access resources in the Azure Portal and create GPT models.

- Save the Azure OpenAI Endpoint URL, Azure OpenAI API Key, and GPT model deployment name (deployment_id) used for creation in a .env file.
- If the .env file does not exist, copy the .env.sample file and rename it.
- The following commands are examples of using the CLI; developers can move on from here [to the next section](./02_OpenAI_getting_started.ipynb).
- Updated on 2024-12-10 (Major model updated to the latest version, code modifications).

# Resource Creation Guide for Prompt Engineering Workshop - CLI Version
This guide is for preparing infrastructure in advance when conducting large-scale Prompt Engineering Workshop training for enterprise-level events.
If the event lasts for more than two days and involves a hackathon with more than 10 developers, this guide covers the resource creation and permission management process.

***The CLI code below is intended for Infra team members preparing the workshop.***
For individuals, there is no need to create resources on such a large scale as shown below.

## 1. Create Subscription and Approve AOAI
- Create an Azure account.
- Create a Subscription (team count = subscriptions * regions).
  - Determine the number of subscriptions needed as follows. Creating multiple subscriptions in advance helps overcome quotas (API limits).
  - For example, if there are 75 participants divided into 15 teams, create 1 subscription and 10 regions (eastus, eastus2, westus3, canadaeast, uksouth, swedencentral, francecentral, norwayeast, japaneast, southindia).
  - Example: 10 teams = 1 subscription * 10 regions, 20 teams = 1 subscription * 10 regions * 2, 30 teams = 1 subscription * 10 regions * 3.
  - If exceeding 30 teams, add more subscriptions as needed.

## 2. Pre-create Resource Groups and Resources per Subscription
- Once OpenAI usage is approved for each subscription, pre-create Resource Groups and resources (Azure OpenAI Resource) to save time during workshops and hackathons.
  - Perform RBAC mapping in advance so that users participating in the event can access designated resources.
  - During the workshop, control service access permissions for participants based on [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles#contributor). After the workshop ends, remove all permissions, adhering to the principle of granting minimal privileges.
  - Use the following regions that support `text-embedding-3-large`, `GPT-4o`, and `GPT-4o mini` simultaneously as of 2024-12-10:
    - eastus, eastus2, westus3, canadaeast, uksouth, swedencentral, francecentral, norwayeast, japaneast, southindia

- **Example of Resource Creation via CLI**
Follow your company’s naming convention. Execute the commands below for each pre-created subscription.
These commands can be initiated by launching the `Cloud Shell (PowerShell)` from the search bar at the top of the Azure Portal.

It is recommended to customize and execute these CLI commands in a Jupyter Notebook afterward.

| Step | Task | CLI Command |
|---|---|---|
|01|Check active tenant|```az account show```|
|02|Set subscription|```az account set --subscription "xxxxxxxxx-xxxxx-xxxx-xxxx-xxxxxxxxxxx"```|
|03|Create resource group|```az group create --name companython-2024-rg --location eastus```|
|04|Create OpenAI region resource 01|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-01 --custom-domain companython-2024-pt-01 --kind OpenAI --sku s0 -l eastus```|
|05|Create OpenAI region resource 02|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-02 --custom-domain companython-2024-pt-02 --kind OpenAI --sku s0 -l eastus2```|
|06|Create OpenAI region resource 03|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-03 --custom-domain companython-2024-pt-03 --kind OpenAI --sku s0 -l westus3```|
|07|Create OpenAI region resource 04|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-04 --custom-domain companython-2024-pt-04 --kind OpenAI --sku s0 -l canadaeast```|
|08|Create OpenAI region resource 05|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-05 --custom-domain companython-2024-pt-05 --kind OpenAI --sku s0 -l uksouth```|
|09|Create OpenAI region resource 06|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-06 --custom-domain companython-2024-pt-06 --kind OpenAI --sku s0 -l swedencentral```|
|10|Create OpenAI region resource 07|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-07 --custom-domain companython-2024-pt-07 --kind OpenAI --sku s0 -l francecentral```|
|11|Create OpenAI region resource 08|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-08 --custom-domain companython-2024-pt-08 --kind OpenAI --sku s0 -l norwayeast```|
|12|Create OpenAI region resource 09|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-09 --custom-domain companython-2024-pt-09 --kind OpenAI --sku s0 -l japaneast```|
|13|Create OpenAI region resource 10|```az cognitiveservices account create -g companython-2024-rg -n companython-2024-pt-10 --custom-domain companython-2024-pt-10 --kind OpenAI --sku s0 -l southindia```|

***Customization Required!*** - Change the prefix names for `prompthon_name` or `prompthon_team_name` per subscription. Ensure these values are globally unique.
If there are multiple subscriptions, repeat step 2 for each new subscription after completing step 3.

In [6]:
company_event_name = "companython-2024"

Below is a guide on logging into Azure CLI from the current device.

In [ ]:
!az config set core.login_experience_v2=off
!az login --use-device-code

In [ ]:
!az account show

***Customization Required!*** - Copy and replace the `id` value of the subscription used for Prompthon below. (If already selected above, this step can be skipped.)

In [ ]:
!az account set --subscription "xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx"

Create a `Resource Group` to manage resources collectively during the workshop or hackathon. All subsequent resources will be managed based on this `Resource Group`.

In [ ]:
!az group create --name $company_event_name"-rg" --location eastus

Create resources for each region per subscription. (Each region’s resource takes about 1 minute to create.)
Below is an example for creating 25 teams, and up to 30 resources can be created per subscription.

In [ ]:
total_resources = 25
region_list = ['eastus', 'eastus2', 'westus3', 'canadaeast', 'uksouth', 'swedencentral', 'francecentral', 'norwayeast', 'japaneast', 'southindia']
region_list = region_list * (total_resources // len(region_list)) + region_list[:total_resources % len(region_list)]

for i in range(0, len(region_list)):
    number = str(i).zfill(2)
    create_openai_resource = (f'az cognitiveservices account create -g {company_event_name}-rg -n {company_event_name}-pt-{number} --custom-domain {company_event_name}-pt-{number} --kind OpenAI --sku s0 -l {region_list[i]}')
    !{create_openai_resource}

If additional subscriptions are available, repeat the above steps for each subscription based on its subscription ID.

## 3. Deploy Azure OpenAI Models
Deploy models to each Azure OpenAI resource created above.
Deploy three models (text-embedding-3-large, gpt-4o, gpt-4o-mini) for each resource (repeat the deployment for all resources created in step 2.4).
Deploying three models per region takes approximately 10 seconds.

| Step | Task | CLI Command |
|---|---|---|
|1|Deploy embedding API|```az cognitiveservices account deployment create --name companython-2024-pt-01 --resource-group companython-2024-rg --deployment-name text-embedding-3-large --model-name text-embedding-3-large --model-version "1" --model-format OpenAI --sku-capacity "100" --sku-name "Standard"```|
|2|Deploy gpt-4o as Global Standard API|```az cognitiveservices account deployment create --name companython-2024-pt-01 --resource-group companython-2024-rg --deployment-name gpt-4og --model-name gpt-4o --model-version "2024-08-06" --model-format OpenAI --sku-capacity "100" --sku-name "GlobalStandard"```|
|3|Deploy gpt-4o mini as Global Standard API|```az cognitiveservices account deployment create --name companython-2024-pt-01 --resource-group companython-2024-rg --deployment-name gpt-4o --model-name gpt-4o --model-version "2024-07-18" --model-format OpenAI --sku-capacity "100" --sku-name "GlobalStandard"```|

In [ ]:
for i in range(0, len(region_list)):
    number = str(i).zfill(2)
    text_embedding_model = (f'az cognitiveservices account deployment create --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name text-embedding-3-large --model-name text-embedding-3-large --model-version "1" --model-format OpenAI --sku-capacity "100" --sku-name "Standard"')
    gpt_4o_model =         (f'az cognitiveservices account deployment create --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name gpt-4o --model-name gpt-4o --model-version "2024-08-06" --model-format OpenAI --sku-capacity "100" --sku-name "GlobalStandard"')
    gpt_4o_mini_model =    (f'az cognitiveservices account deployment create --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name gpt-4o-mini --model-name gpt-4o-mini --model-version "2024-07-18" --model-format OpenAI --sku-capacity "100" --sku-name "GlobalStandard"')
    !{text_embedding_model}
    !{gpt_4o_model}
    !{gpt_4o_mini_model}

## 4. Register Users
Register users in Microsoft Entra ID. For ease of management, create groups per team or hackathon group.
Assign the registered users to these groups.
- Use the following CLI command to register users: [Reference Documentation](https://learn.microsoft.com/en-us/cli/azure/ad/user?view=azure-cli-latest#az-ad-user-create).
```az ad user create --display-name myuser --password password --user-principal-name myuser@contoso.com```

## 5. Granting Resource Permissions
In addition to user registration, register participants in Microsoft Entra ID and grant them access to Azure OpenAI, AI Search, Blob Storage, and AI Foundry.  
For quick experimentation and testing during the hackathon, it is recommended to assign ["Contributor"](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles#contributor) role to participants for the subscription. Group-based management is also possible.

Following the principle of granting minimal privileges is considered a best practice; ensure to revoke permissions immediately after the hackathon ends.  
If you want to configure access based on **minimum privilege security best practices**, follow these steps:  
- Go to the IAM of the Resource Groups created in steps 2–3 and assign the following roles to the respective groups (OpenAI, AI Search, Storage Account permissions).  
- If you want to use the Add your data feature in Azure OpenAI Studio to connect RAG services (AI Search and Blob Storage connection), it is recommended to register the subscription ID with the Contributor role. This avoids the cumbersome process of repeatedly setting permissions for Blob Storage every time an AI Search service is created. [Learn more about the structure](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/use-your-data-securely).

By following the above method, Azure services can be used for practice without granting Owner permissions to users or groups at the subscription level.  
If you want to use other services such as OCR, Text-to-speech, Speech-to-Text, App Services (instant chatbot service deployment), or Promptflow, you can assign the necessary roles for these services individually. However, it is worth noting that handling permission requests from dozens of users on the day of the hackathon can be cumbersome.  
  
Once this process is complete, ensure that users can access the resources and activate the services by logging in beforehand and testing their usage in Azure OpenAI Playground. This preparation is ideal if there is a 1–2 week period from subscription creation to user access testing.

## 6. Resource Deletion Method
Resources should be deleted in reverse order of creation.  
Before deleting resources, it is recommended to delete the models deployed in the Azure OpenAI resources and then delete the Resource Groups one by one.  
If you delete resources without deleting the models first, there may be issues where quotas for deployed models remain as if they are still allocated.

### 6.1 Deleting Deployed Azure OpenAI Models

In [ ]:
for i in range(0, len(region_list)):
    number = str(i).zfill(2)
    text_embedding_model = (f'az cognitiveservices account deployment delete --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name text-embedding-3-large')
    gpt_4o_model =         (f'az cognitiveservices account deployment delete --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name gpt-4o')
    gpt_4o_mini_model =    (f'az cognitiveservices account deployment delete --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name gpt-4o-mini')
    !{text_embedding_model}
    !{gpt_4o_model}
    !{gpt_4o_mini_model}

### 6.2 Deleting Deployed Azure OpenAI Resources
Deleting resources per region takes approximately 10 seconds.  
(Ensure the models are deleted before deleting the resources.)

In [ ]:
for i in range(0, len(region_list)):
    number = str(i).zfill(2)
    delete_openai_resource = (f'az cognitiveservices account delete -g {company_event_name}-rg -n {company_event_name}-pt-{number}')
    !{delete_openai_resource}

### 6.1 Deleting Deployed Azure OpenAI Models

In [ ]:
for i in range(0, len(region_list)):
    number = str(i).zfill(2)
    text_embedding_model = (f'az cognitiveservices account deployment delete --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name text-embedding-3-large')
    gpt_4o_model =         (f'az cognitiveservices account deployment delete --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name gpt-4o')
    gpt_4o_mini_model =    (f'az cognitiveservices account deployment delete --name {company_event_name}-pt-{number} --resource-group {company_event_name}-rg --deployment-name gpt-4o-mini')
    !{text_embedding_model}
    !{gpt_4o_model}
    !{gpt_4o_mini_model}

### 6.2 Deleting Deployed Azure OpenAI Resources
Deleting resources per region takes approximately 10 seconds. (Ensure the models are deleted before deleting the resources.)

In [ ]:
for i in range(0, len(region_list)):
    number = str(i).zfill(2)
    delete_openai_resource = (f'az cognitiveservices account delete -g {company_event_name}-rg -n {company_event_name}-pt-{number}')
    !{delete_openai_resource}